# This is a trial for onnx to convert tensorflow model

In [1]:
import tensorflow as tf
from tensorflow import keras
import tf2onnx
import numpy as np
import onnxruntime as ort

/home/debo/envOnnxPy3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/debo/envOnnxPy3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/debo/envOnnxPy3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/debo/envOnnxPy3/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.

In [2]:
tf.__version__

'1.14.0'

In [3]:
tf2onnx.__version__

'1.5.3'

In [4]:
ort.__version__

'0.4.0'

# A simple MNIST handwritten digit classification using tensorFlow

In [5]:
mnist = tf.keras.datasets.mnist

(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

In [6]:
import matplotlib.pyplot as plt

In [7]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

W0928 17:34:53.615011 140640642094848 deprecation.py:506] From /home/debo/envOnnxPy3/lib/python3.5/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [8]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.fit(x_train, y_train, epochs=5)

Epoch 1/5
60000/60000 [==============================] - 7s 110us/sample - loss: 0.2190 - acc: 0.9350
Epoch 2/5
60000/60000 [==============================] - 7s 111us/sample - loss: 0.0971 - acc: 0.9700
Epoch 3/5
60000/60000 [==============================] - 7s 109us/sample - loss: 0.0703 - acc: 0.9783
Epoch 4/5
60000/60000 [==============================] - 7s 112us/sample - loss: 0.0542 - acc: 0.9828
Epoch 5/5
60000/60000 [==============================] - 6s 107us/sample - loss: 0.0425 - acc: 0.9857


In [10]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 0s 45us/sample - loss: 0.0641 - acc: 0.9801


[0.06414145312517648, 0.9801]

In [11]:
prediction = model.predict(x_test)

In [12]:
print(prediction[1])

[1.6688519e-11 2.6992730e-05 9.9997258e-01 4.8012748e-07 1.1671214e-15
 2.8190819e-10 1.5331255e-11 1.9634591e-13 4.8831972e-10 4.7369840e-14]


In [34]:
print(y_test[3])

0


In [15]:
print("the digit is classified as \"%s\" in prediction"%np.argmax(prediction[1]))

the digit is classified as "2" in prediction


In [16]:
saved_model_path = "./saved_mnist"

In [17]:
keras.experimental.export_saved_model(model, saved_model_path)

W0928 17:37:08.553953 140640642094848 deprecation.py:506] From /home/debo/envOnnxPy3/lib/python3.5/site-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0928 17:37:08.557308 140640642094848 deprecation.py:506] From /home/debo/envOnnxPy3/lib/python3.5/site-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0928 17:37:09.315115 140640642094848 deprecation.py:323] From /home/debo/envOnnxPy3/lib/python3.5/site-packages/tensorflow/python/saved_model/signature_def_utils_impl.py:253: build_tensor_info (from 

# Converting .pb model to .onnx and verify the prediction using ONNXRruntime

In [27]:
#python -m tf2onnx.convert --saved-model ./saved_mnist/ --output ./saved_mnist/saved_model.onnx

In [18]:
import numpy as np
import onnxruntime as ort

In [19]:
sess_ort = ort.InferenceSession("./saved_mnist/saved_model.onnx")

In [20]:
input_name = sess_ort.get_inputs()[0].name
print("input name", input_name)
input_shape = sess_ort.get_inputs()[0].shape
print("input shape", input_shape)
input_type = sess_ort.get_inputs()[0].type
print("input type", input_type)

input name flatten_input:0
input shape [None, 28, 28]
input type tensor(float)


In [21]:
output_name = sess_ort.get_outputs()[0].name
print("output_name", output_name)
output_shape = sess_ort.get_outputs()[0].shape
print("output shape", output_shape)
output_type = sess_ort.get_outputs()[0].type
print("output type", output_type)

output_name dense_1/Softmax:0
output shape [None, 10]
output type tensor(float)


# Using a test sample for validation

In [38]:
img = x_test[0]
img = img.astype(np.float32)

In [39]:
img=img.reshape(1,28,28)
img.shape

(1, 28, 28)

In [40]:
res = sess_ort.run([output_name], {input_name: img})

In [41]:
print("the digit is classified as \"%s\" in ONNXRruntime"%np.argmax(res))

the digit is classified as "7" in ONNXRruntime
